# Unsupervised Clustering using VQE

### DataPoint
First, we'll have to create a DataPoint class, that can be used to store and compare input-datapoints. We'll do this as follows, including a ```dist``` function to compute the squared-distance between two datapoints:

In [ ]:
from dataclasses import dataclass
from typing import List
import numpy as np


# Helper functions
def sqr(x: float):
    return x * x


@dataclass
class DataPoint:
    x: float
    y: float
    cluster: int = 0
    original_cluster: int = 0
    fidelity: float = 0.0

    def dist(self, o: "DataPoint") -> float:
        return abs(o.x - self.x) + abs(o.y - self.y)


@dataclass
class Dataset:
    _clusters: List[List[DataPoint]]
    min_x: int = 0
    max_x: int = 0
    min_y: int = 0
    max_y: int = 0
    _complete = None

    cached_distance_matrix = None

    def __len__(self):
        return len(self.complete_dataset)

    @property
    def accuracy(self) -> float:
        # Accuracy metric: Correctly classified to the initial cluster / total
        total = 0
        correct = 0
        for cluster in self._clusters:
            for point in cluster:
                total += 1
                if point.original_cluster == point.cluster:
                    correct += 1

        return correct / total

    def __getitem__(self, item):
        return self.complete_dataset[item]

    @property
    def complete_dataset(self) -> List[DataPoint]:
        if self._complete is not None:
            return self._complete

        # normalize clusters
        for i in self._clusters:
            for j in i:
                j.x = (j.x - self.min_x) / (self.max_x - self.min_x)
                j.y = (j.y - self.min_y) / (self.max_y - self.min_y)
        # Compute the complete dataset and cache it
        self._complete = [point for cluster in self._clusters for point in cluster]
        return self._complete

    @staticmethod
    def gen_clusters(**kwargs) -> "Dataset":
        clusters = []

        min_x = np.inf
        min_y = np.inf

        max_x = -np.inf
        max_y = -np.inf

        for i in range(1, kwargs.get("clusters", 2) + 1):
            current_cluster = []
            # Load cluster parameters
            cluster_angle_start = kwargs.get(f"cluster_{i}_angle_start", 0)
            cluster_angle_end = kwargs.get(f"cluster_{i}_angle_end", 360)
            cluster_center_x = kwargs.get(f"cluster_{i}_center_x", 0)
            cluster_center_y = kwargs.get(f"cluster_{i}_center_y", 0)
            cluster_radius_start = kwargs.get(f"cluster_{i}_radius_start", 0)
            cluster_radius_end = kwargs.get(f"cluster_{i}_radius_end", 50)

            # Compute normalization terms
            min_x = min(min_x, cluster_center_x - cluster_radius_end)
            min_y = min(min_y, cluster_center_y - cluster_radius_end)

            max_x = max(max_x, cluster_center_x + cluster_radius_end)
            max_y = max(max_y, cluster_center_y + cluster_radius_end)

            # Generate the needed points
            for j in range(kwargs.get(f"cluster_{i}_size", 15)):
                angle = np.deg2rad(np.random.uniform(cluster_angle_start, cluster_angle_end))
                radius = np.random.uniform(cluster_radius_start, cluster_radius_end)
                x = cluster_center_x + radius * np.cos(angle)
                y = cluster_center_y + radius * np.sin(angle)
                current_cluster.append(DataPoint(x, y, original_cluster=i - 1, cluster=i - 1))

            clusters.append(current_cluster)

        return Dataset(_clusters=clusters, min_x=min_x, min_y=min_y, max_x=max_x, max_y=max_y)

    @property
    def distance_matrix(self) -> np.array:
        """
        Distance matrix[i][j] = distance between i and j
        :return:
        """
        if self.cached_distance_matrix is not None:
            return self.cached_distance_matrix
        complete_dataset: List[DataPoint] = self.complete_dataset
        dataset_size = len(complete_dataset)
        distance_matrix = np.zeros(shape=(dataset_size, dataset_size))
        for i in range(dataset_size):
            for j in range(dataset_size):
                distance_matrix[i][j] = complete_dataset[i].dist(complete_dataset[j])
        self.cached_distance_matrix = distance_matrix
        return distance_matrix

    @staticmethod
    def blobs(cluster_1_size=100, cluster_2_size=100) -> "Dataset":
        """Generates points in blobs around the 20 and 80 point"""
        return Dataset.gen_clusters(
            clusters=2,
            cluster_1_size=cluster_1_size,
            cluster_1_center_x=20,
            cluster_1_center_y=50,
            cluster_1_radius_end=20,

            cluster_2_size=cluster_2_size,
            cluster_2_center_x=80,
            cluster_2_center_y=50,
            cluster_2_radius_end=20
        )

    @staticmethod
    def concave_clusters(cluster_1_size=100, cluster_2_size=100) -> "Dataset":
        return Dataset.gen_clusters(clusters=2,
                                    cluster_1_size=cluster_1_size,
                                    cluster_1_angle_start=0, cluster_1_angle_end=180,
                                    cluster_1_center_x=45, cluster_1_center_y=50,
                                    cluster_1_radius_start=8, cluster_1_radius_end=10,

                                    cluster_2_size=cluster_2_size,
                                    cluster_2_angle_start=180, cluster_2_angle_end=360,
                                    cluster_2_center_x=55, cluster_2_center_y=50,
                                    cluster_2_radius_start=8, cluster_2_radius_end=10, )

    def plot(self, initial=False):
        clusters = [list() for i in self._clusters]
        for i in self.complete_dataset:
            clusters[i.cluster if not initial else i.original_cluster].append(i)

        for cluster in clusters:
            plt.scatter(
                [point.x for point in cluster],
                [point.y for point in cluster]
            )

        plt.show()

Also implement the ```map``` function, that can be used to map Data Points onto the Bloch Sphere:

In [ ]:
def map(a: float, b: float, c: float, d: float, e: float) -> float:
    return d + ((a - b) / (c - b)) * (e - d)

### Qiskit
Now, we start working with Qiskit! Import all the necessary elements:

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

from qiskit import Aer
from qiskit import QuantumCircuit
from qiskit import transpile
from qiskit.algorithms.optimizers import SPSA
from qiskit.quantum_info import state_fidelity

from tqdm import tqdm
import math

In [ ]:
dataset = Dataset.blobs(100, 100)
dataset.plot()
dataset.distance_matrix


In [ ]:
# Fix the np random seed
np.random.seed(12)

#
# Hard-code the data points that the algorithm will be performed on
#

# Hard-code the two reference states on the bloch-sphere that we will use as references for two clusters
#
# [1, 0] = |0> and [0, 1] = |1>
reference_points = [[1, 0], [0, 1]]

circuit_counter = 0


# Function: setup the variational form according to given parameters


def setup_variational_form(dp: DataPoint, params) -> QuantumCircuit:
    global circuit_counter

    q_circuit = QuantumCircuit(1)
    q_circuit.name = "circuit" + str(circuit_counter)
    circuit_counter += 1
    q_circuit.initialize([1, 0], 0)

    q_circuit.u(
        map(dp.x, 0, 1, 0, math.pi),
        map(dp.y, 0, 1, 0, math.pi),
        0,
        0
    )
    q_circuit.u(params[0], params[1], 0, 0)

    #print("Quantum Circuit:")
    #print(q_circuit)

    q_circuit.save_statevector()

    return q_circuit


#setup_variational_form(DataPoint(0, 0), [0, 0])

qiskit_backend = Aer.get_backend("aer_simulator")


def run_circuit_for_point(data_points, data_point_index: int, params):
    return qiskit_backend.run(
        transpile(
            setup_variational_form(data_points[data_point_index], params),
            backend=qiskit_backend
        )
    ).result().get_statevector()


def objective_function_with_datapoints(data_points: Dataset):
    def objective(params) -> float:

        state_lambda = lambda x: run_circuit_for_point(data_points, x, params)

        n = len(data_points)
        m = len(reference_points)
        # Computing state fidelity
        state_cache = [state_lambda(i) for i in tqdm(range(n))]
        # Computing the fidelity matrix f(i, a) = fidelity between datapoint i and reference a
        fidelity_matrix = np.zeros(shape=(n, m))
        for i in range(n):
            for j in range(m):
                fidelity_matrix[i][j] = state_fidelity(state_cache[i], reference_points[j])

        # Computing the fidelity matrix f(i, j) = sum over all references [ f(i, a) * f(j, a) ]
        fidelity_sum_matrix = np.zeros(shape=(n, n))
        for i in range(n):
            for j in range(m):
                if i == j: continue
                if j < i:
                    fidelity_sum_matrix[i][j] = fidelity_sum_matrix[j][i]
                else:
                    fidelity_sum = np.dot(fidelity_matrix[i, :], fidelity_matrix[j, :])
                    fidelity_sum_matrix[i][j] = fidelity_sum

        # Cost function = sum over all distance[i][j] * fidelity_sum_matrix[i][j]
        cost = np.sum(np.multiply(data_points.distance_matrix, fidelity_sum_matrix))
        return cost

    return objective


# Select the right optimizer
optimizer = SPSA(maxiter=2)


def cluster_dataset(dataset: Dataset):
    # Initialize the parameters with random values
    params_init = np.random.rand(2)
    fun = objective_function_with_datapoints(dataset)
    # Perform the optimization and store the result
    optimal_params = optimizer.minimize(fun=fun, x0=params_init).x

    print(optimal_params)
    print("Total cost: " + str((fun(optimal_params))))

    n = len(dataset)
    m = len(reference_points)
    state_vectors = [run_circuit_for_point(dataset, i, optimal_params) for i in range(n)]
    fidelity_matrix = np.zeros(shape=(n, m))
    for i in range(n):
        for j in range(m):
            fidelity_matrix[i][j] = state_fidelity(state_vectors[i], reference_points[j])

    cluster_iter = iter(np.argmax(fidelity_matrix, axis=1))
    for i in dataset.complete_dataset:
        i.cluster = next(cluster_iter)
    dataset.plot(initial=True)
    dataset.plot()
    print(f"Accuracy: {dataset.accuracy}")


In [ ]:
cluster_dataset(Dataset.blobs(100, 100))

In [ ]:
cluster_dataset(Dataset.concave_clusters(100, 100))